## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

# check Teh GPU 

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

# Colab connection with the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **27.01.2023** with version **YOLOv8.0.20**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

# Install ththe yolo8 librart from ultralytics

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
!Git clone method (for development)

#%cd {HOME}
!git clone github.com/ultralytics/ultralytics
%cd {HOME}/ultralytics
!pip install -e .

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

# This Augemntation code is for bounding box with 4 co-ordinates 

#Augment the images using the code

In [ ]:
import cv2
import os
import math
import numpy as np

#def histogram_equalization(image):
    # Convert image to grayscale
 #   gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply histogram equalization
  #  equalized = cv2.equalizeHist(gray)

    # Convert back to BGR color space
   # equalized_image = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)

    #return equalized_image

#def adjust_brightness(image, value):
    # Convert image to HSV color space
 #   hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Adjust brightness
  #  hsv[..., 2] = cv2.add(hsv[..., 2], value)

    # Convert back to BGR color space
   # brightness_adjusted = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    #return brightness_adjusted

def horizontal_flip(image, annotations):
    flipped = cv2.flip(image, 1)
    flipped_annotations = [(class_id, 1 - x, y, width, height) for class_id, x, y, width, height in annotations]
    return flipped, flipped_annotations

def vertical_flip(image, annotations):
    flipped = cv2.flip(image, 0)
    flipped_annotations = [(class_id, x, 1 - y, width, height) for class_id, x, y, width, height in annotations]
    return flipped, flipped_annotations


def parse_yolo_annotation(annot_path):
    with open(annot_path, 'r') as f:
        lines = f.read().splitlines()

    annotations = []
    for line in lines:
        class_id, x, y, width, height = map(float, line.split())
        annotations.append((class_id, x, y, width, height))

    return annotations

def save_yolo_annotation(annot_path, annotations):
    with open(annot_path, 'w') as f:
        for annotation in annotations:
            class_id, x, y, width, height = annotation
            line = f"{int(class_id)} {x} {y} {width} {height}"
            f.write(line + '\n')

# Define the input directory containing the original images
input_dir = '/home/buddhad'

# Define the input directory containing the original YOLO annotations
annotation_dir = '/home/buddhadev/Downloads/Nair/annotations_corrected/labels'

# Define the output directory to save augmented images
output_dir = '/home/buddhadev/Downloads/Nair/annotations_corrected/aug_images'
os.makedirs(output_dir, exist_ok=True)

# Define the output directory to save augmented YOLO annotations
aug_annotation_dir = '/home/buddhadev/Downloads/Nair/annotations_corrected/aug_lables'
os.makedirs(aug_annotation_dir, exist_ok=True)

# Define the categories and corresponding labels
categories = [ 'horizontal_flip', 'vertical_flip' ]



# Process each category
for category in categories:
    # Create a subdirectory for the category
    category_dir = os.path.join(output_dir, category)
    os.makedirs(category_dir, exist_ok=True)

    # Create a subdirectory for the augmented annotations
    aug_category_annot_dir = os.path.join(aug_annotation_dir, category)
    os.makedirs(aug_category_annot_dir, exist_ok=True)

    # Process each image in the input directory
    for filename in os.listdir(input_dir):
        image_path = os.path.join(input_dir, filename)

        # Read the image
        image = cv2.imread(image_path)

        # Read the original annotation file
        annotation_path = os.path.join(annotation_dir, f"{os.path.splitext(filename)[0]}.txt")
        original_annotations = parse_yolo_annotation(annotation_path)

        if  category == 'horizontal_flip':
            # Apply horizontal flip
            augmented_image, augmented_annotations = horizontal_flip(image, original_annotations)
            # Update the filename and label
            augmented_filename = f"hor_flip_{filename}"
            label = f"{filename}_hor_flip"
        elif category == 'vertical_flip':
            # Apply vertical flip
            augmented_image, augmented_annotations = vertical_flip(image, original_annotations)
            # Update the filename and label
            augmented_filename = f"ver_flip_{filename}"
            label = f"{filename}_ver_flip"

              # Calculate the new image dimensions
            image_height, image_width, _ = augmented_image.shape
        else:
            continue

        # Save the augmented image
        augmented_image_path = os.path.join(category_dir, augmented_filename)
        cv2.imwrite(augmented_image_path, augmented_image)

        # Save the corresponding label file
        label_file_path = os.path.join(aug_category_annot_dir, f"{os.path.splitext(augmented_filename)[0]}.txt")
        with open(label_file_path, 'w') as f:
            f.write(label)

        # Save the corresponding augmented annotation file
        aug_annotation_path = os.path.join(aug_category_annot_dir, f"{os.path.splitext(augmented_filename)[0]}.txt")
        save_yolo_annotation(aug_annotation_path, augmented_annotations)

# This code is augmentation of Polygon whenre the co-ordianes are more than 4 

In [ ]:
import cv2
import os
import numpy as np

def horizontal_flip(image, annotations):
    flipped = cv2.flip(image, 1)
    flipped_annotations = [(class_id, [(1 - x, y) for x, y in points]) for class_id, points in annotations]
    return flipped, flipped_annotations

def vertical_flip(image, annotations):
    flipped = cv2.flip(image, 0)
    flipped_annotations = [(class_id, [(x, 1 - y) for x, y in points]) for class_id, points in annotations]
    return flipped, flipped_annotations

def parse_yolo_annotation(annot_path):
    with open(annot_path, 'r') as f:
        lines = f.read().splitlines()

    annotations = []
    for line in lines:
        values = line.split()
        class_id = int(values[0])
        points = [(float(values[i]), float(values[i+1])) for i in range(1, len(values)-1, 2)]
        annotations.append((class_id, points))

    return annotations

def save_yolo_annotation(annot_path, annotations):
    with open(annot_path, 'w') as f:
        for annotation in annotations:
            class_id, points = annotation
            points_str = " ".join([f"{x} {y}" for x, y in points])
            line = f"{int(class_id)} {points_str}"
            f.write(line + '\n')

input_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/YOLO_OPMD_SP/images'
annotation_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/YOLO_OPMD_SP/labels'
output_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/aug_images'
os.makedirs(output_dir, exist_ok=True)
aug_annotation_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/aug_labels'
os.makedirs(aug_annotation_dir, exist_ok=True)
categories = ['horizontal_flip', 'vertical_flip']

for category in categories:
    category_dir = os.path.join(output_dir, category)
    os.makedirs(category_dir, exist_ok=True)
    aug_category_annot_dir = os.path.join(aug_annotation_dir, category)
    os.makedirs(aug_category_annot_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        image_path = os.path.join(input_dir, filename)
        image = cv2.imread(image_path)
        annotation_path = os.path.join(annotation_dir, f"{os.path.splitext(filename)[0]}.txt")
        original_annotations = parse_yolo_annotation(annotation_path)

        if category == 'horizontal_flip':
            augmented_image, augmented_annotations = horizontal_flip(image, original_annotations)
            augmented_filename = f"hor_flip_{filename}"
        elif category == 'vertical_flip':
            augmented_image, augmented_annotations = vertical_flip(image, original_annotations)
            augmented_filename = f"ver_flip_{filename}"
        else:
            continue

        augmented_image_path = os.path.join(category_dir, augmented_filename)
        cv2.imwrite(augmented_image_path, augmented_image)
        aug_annotation_path = os.path.join(aug_category_annot_dir, f"{os.path.splitext(augmented_filename)[0]}.txt")
        save_yolo_annotation(aug_annotation_path, augmented_annotations)


In [ ]:
import os
import shutil

# Function to copy files from source folder to destination folder
def copy_files(source_folder, destination_folder):
    for filename in os.listdir(source_folder):
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)
        shutil.copyfile(source_path, destination_path)

# List of image folders and label folders
image_folders = [
    "/home/buddhadev/Buddhadev_Everything/OPMD/YOLO_OPMD_SP/images",
    "/home/buddhadev/Buddhadev_Everything/OPMD/aug_images/horizontal_flip",
    "/home/buddhadev/Buddhadev_Everything/OPMD/aug_images/vertical_flip"
    

]

label_folders = [
   "/home/buddhadev/Buddhadev_Everything/OPMD/YOLO_OPMD_SP/labels",
    "/home/buddhadev/Buddhadev_Everything/OPMD/aug_labels/horizontal_flip",
    "/home/buddhadev/Buddhadev_Everything/OPMD/aug_labels/vertical_flip"

]

# Create destination folders
augmented_images_folder = "/home/buddhadev/Buddhadev_Everything/OPMD/AUGEMENTATION/combined_images"
augmented_labels_folder = "/home/buddhadev/Buddhadev_Everything/OPMD/AUGEMENTATION/combined_labels"
os.makedirs(augmented_images_folder, exist_ok=True)
os.makedirs(augmented_labels_folder, exist_ok=True)

# Copy images and labels to destination folders
for folder in image_folders:
    copy_files(folder, augmented_images_folder)

for folder in label_folders:
    copy_files(folder, augmented_labels_folder)

print("Images and labels have been combined and stored in 'augmentedimagesnew' and 'augmentedlabelsnew' respectively.")

## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

### 💻 CLI

`yolo mode=predict` runs YOLOv8 inference on a variety of sources, downloading models automatically from the latest YOLOv8 release, and saving results to `runs/predict`.

### 🐍 Python SDK

The simplest way of simply using YOLOv8 directly in a Python environment.

In [ ]:
import os
import random
import shutil

def split_data(images_dir, labels_dir, split_ratio):
    # Create folders for train, test, and valid sets
    os.makedirs('train/images', exist_ok=True)
    os.makedirs('train/labels', exist_ok=True)
    os.makedirs('test/images', exist_ok=True)
    os.makedirs('test/labels', exist_ok=True)
    os.makedirs('valid/images', exist_ok=True)
    os.makedirs('valid/labels', exist_ok=True)

    # Get a list of all image files
    image_files = os.listdir(images_dir)

    # Shuffle the image file list randomly
    random.shuffle(image_files)

    # Calculate the number of files for each set based on the split ratio
    num_images = len(image_files)
    num_train = int(num_images * split_ratio[0])
    num_test = int(num_images * split_ratio[1])
    num_valid = num_images - num_train - num_test

    # Move files to their respective sets
    for i, image_file in enumerate(image_files):
        # Generate corresponding label file name
        label_file = os.path.splitext(image_file)[0] + '.txt'

        if i < num_train:
            shutil.copy(os.path.join(images_dir, image_file), 'train/images')
            shutil.copy(os.path.join(labels_dir, label_file), 'train/labels')
        elif i < num_train + num_test:
            shutil.copy(os.path.join(images_dir, image_file), 'test/images')
            shutil.copy(os.path.join(labels_dir, label_file), 'test/labels')
        else:
            shutil.copy(os.path.join(images_dir, image_file), 'valid/images')
            shutil.copy(os.path.join(labels_dir, label_file), 'valid/labels')

# Example usage
split_ratio = [0.8, 0.2]  # Train: 70%, Test: 15%, Valid: 15%
images_dir = "/home/buddhadev/Buddhadev_Everything/OPMD/AUGEMENTATION/combined_images"
labels_dir = "/home/buddhadev/Buddhadev_Everything/OPMD/AUGEMENTATION/combined_labels"

split_data(images_dir, labels_dir, split_ratio)

In [ ]:
import zipfile
import os

# Specify the directory containing the files you want to zip
directory = '/content/drive/MyDrive/google_images'

# Create a zip file in write mode
zip_name = 'files.zip'
zip_path = '/content/' + zip_name
zipf = zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED)

# Iterate over all files in the directory and add them to the zip file
for foldername, subfolders, filenames in os.walk(directory):
    for filename in filenames:
        file_path = os.path.join(foldername, filename)
        zipf.write(file_path, os.path.relpath(file_path, directory))

# Close the zip file
zipf.close()

# Download the zip file
from google.colab import files
files.download(zip_path)

# Split the data into test train and valid

In [ ]:
`

    # Calculate the number of files for each set
    num_images = len(image_files)
    num_train = int(num_images * 0.7)
    num_valid = int(num_images * 0.15)
    num_test = num_images - num_train - num_valid

    # Move files to their respective sets
    for i, image_file in enumerate(image_files):
        # Generate corresponding label file name
        label_file = os.path.splitext(image_file)[0] + '.txt'

        if i < num_train:
            shutil.copy(os.path.join(images_dir, image_file), 'train/images')
            shutil.copy(os.path.join(labels_dir, label_file), 'train/labels')
        elif i < num_train + num_valid:
            shutil.copy(os.path.join(images_dir, image_file), 'valid/images')
            shutil.copy(os.path.join(labels_dir, label_file), 'valid/labels')
        else:
            shutil.copy(os.path.join(images_dir, image_file), 'test/images')
            shutil.copy(os.path.join(labels_dir, label_file), 'test/labels')

# Example usage
images_dir = '/content/newaugimage'
labels_dir = '/content/newauglabel'

split_data(images_dir, labels_dir)


## Custom Training

In [ ]:
#%cd {HOME}

!yolo task=detect mode=train model=yolov8n.pt data=/home/buddhadev/Buddhadev_Everything/OPMD/data.yaml epochs=500 imgsz=640 plots=True patience=500 batch=32


In [ ]:
!ls /content/runs/detect/train/

In [ ]:
#%cd {HOME}
Image(filename=f'/home/buddhadev/Buddhadev_Everything/OPMD/runs/detect/train2/confusion_matrix.png', width=600)

In [ ]:
#%cd {HOME}
Image(filename=f'/home/buddhadev/Buddhadev_Everything/OPMD/runs/detect/train2/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'/content/runs/segment/val/val_batch0_pred.jpg', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'/content/runs/segment/train/train_batch2.jpg', width=600)

## Validate Custom Model

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model=/home/buddhadev/Buddhadev_Everything/OPMD/runs/detect/train6/weights/best.pt data=data.yaml source=data.yaml

In [ ]:
model = YOLO("/home/buddhadev/Buddhadev_Everything/OPMD/runs/detect/train2/weights/best.pt")
model.train(data="data.yaml", epochs=5)
model.val()

## Inference with Custom Model

In [ ]:
#%cd {HOME}
!yolo task=detect mode=predict model=/home/buddhadev/Buddhadev_Everything/OPMD/runs/detect/train2/weights/best.pt  conf=0.25 source=/home/buddhadev/Buddhadev_Everything/OPMD/test/images save=True plots=True

**NOTE:** Let's take a look at few results.

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

# zip the content 

In [ ]:
import zipfile
import os

# Specify the directory containing the files you want to zip
directory = '/content/runs'

# Create a zip file in write mode
zip_name = 'files.zip'
zip_path = '/content/' + zip_name
zipf = zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED)

# Iterate over all files in the directory and add them to the zip file
for foldername, subfolders, filenames in os.walk(directory):
    for filename in filenames:
        file_path = os.path.join(foldername, filename)
        zipf.write(file_path, os.path.relpath(file_path, directory))

# Close the zip file
zipf.close()

# Download the zip file
from google.colab import files
files.download(zip_path)

## Deploy model on Roboflow

Once you have finished training your YOLOv8 model, you’ll have a set of trained weights ready for use. These weights will be in the `/runs/detect/train/weights/best.pt` folder of your project. You can upload your model weights to Roboflow Deploy to use your trained weights on our infinitely scalable infrastructure.

The `.deploy()` function in the [Roboflow pip package](https://docs.roboflow.com/python) now supports uploading YOLOv8 weights.

To upload model weights, add the following code to the “Inference with Custom Model” section in the aforementioned notebook:

In [ ]:
#While your deployment is processing, checkout the deployment docs to take your model to most destinations https://docs.roboflow.com/inference

In [ ]:
#Run inference on your model on a persistant, auto-scaling, cloud API

#load model
model = project.version(dataset.version).model

#choose random test set image
import os, random
test_set_loc = dataset.location + "/test/images/"
random_test_image = random.choice(os.listdir(test_set_loc))
print("running inference on " + random_test_image)

pred = model.predict(test_set_loc + random_test_image, confidence=40, overlap=30).json()
pred

In [ ]:
import os
from PIL import Image

def upscale_image(input_path, output_path, scale_factor=2, method=Image.BICUBIC):
    """
    Upscale an image using the given method.
    
    Parameters:
    - input_path (str): Path to the input image.
    - output_path (str): Path to save the upscaled image.
    - scale_factor (int): Factor by which to upscale the image.
    - method (int): Interpolation method. Default is Image.BICUBIC.
    """
    with Image.open(input_path) as img:
        # Calculate new dimensions
        width, height = img.size
        new_dimensions = (width * scale_factor, height * scale_factor)
        
        # Resize the image
        upscaled_img = img.resize(new_dimensions, method)
        upscaled_img.save(output_path)

def upscale_all_images_in_folder(input_folder, output_folder, scale_factor=2, method=Image.BICUBIC):
    """
    Upscale all images in a folder using the given method.
    
    Parameters:
    - input_folder (str): Path to the input folder containing images.
    - output_folder (str): Path to save the upscaled images.
    - scale_factor (int): Factor by which to upscale the image.
    - method (int): Interpolation method. Default is Image.BICUBIC.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Iterate over all files in the directory
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            
            upscale_image(input_path, output_path, scale_factor, method)

if __name__ == "__main__":
    input_directory = input("Enter the path to the input folder containing images: ")
    output_directory = input("Enter the path where upscaled images should be saved: ")
    scale = int(input("Enter the scale factor (e.g., 2 for doubling the resolution): "))
    
    upscale_all_images_in_folder(input_directory, output_directory, scale)



In [ ]:
import os

def has_two_in_first_column(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            columns = line.strip().split()  # Assuming space-separated columns
            if columns and columns[0] == '2':
                return True
    return False

def main(directory_path):
    txt_files = [f for f in os.listdir(directory_path) if f.endswith('.txt')]
    files_with_two = []

    for txt_file in txt_files:
        file_path = os.path.join(directory_path, txt_file)
        if has_two_in_first_column(file_path):
            files_with_two.append(txt_file)

    return files_with_two

if __name__ == '__main__':
    directory = input('Enter the path of the directory: ')
    result = main(directory)
    if result:
        print('Files with number 2 in the first column:')
        for filename in result:
            print(filename)
    else:
        print('No such files found.')



In [ ]:
import os

def replace_two_with_zero(file_path):
    lines_to_write = []
    replaced = False

    with open(file_path, 'r') as f:
        for line in f:
            columns = line.strip().split()  # Assuming space-separated columns

            if columns and columns[0] == '2':
                columns[0] = '0'
                replaced = True

            lines_to_write.append(" ".join(columns))

    if replaced:  # Only write back to the file if a replacement occurred
        with open(file_path, 'w') as f:
            f.write("\n".join(lines_to_write))

def main(directory_path):
    txt_files = [f for f in os.listdir(directory_path) if f.endswith('.txt')]

    for txt_file in txt_files:
        file_path = os.path.join(directory_path, txt_file)
        replace_two_with_zero(file_path)

if __name__ == '__main__':
    directory = input('Enter the path of the directory: ')
    main(directory)
    print('Processing complete. Checked and replaced if necessary.')



In [ ]:
import os

def rename_files_in_directory(directory_path):
    try:
        # List all files in the specified directory
        files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

        for file_name in files:
            # Check if the file name contains a dash
            if '-' in file_name:
                # Split the file name at the dash and keep only the portion after the dash
                new_file_name = file_name.split('-')[-1]

                # Construct the full path for the old and new file names
                old_file_path = os.path.join(directory_path, file_name)
                new_file_path = os.path.join(directory_path, new_file_name)

                # Rename the file
                os.rename(old_file_path, new_file_path)
                print(f'Renamed: {file_name} -> {new_file_name}')

    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the directory path where the files are located
directory_path = '/home/buddhadev/Buddhadev_Everything/OPMD/YOLO_OPMD_SP/labels'  # <-- REPLACE with your directory path
rename_files_in_directory(directory_path)


In [ ]:
import os
import shutil
from glob import glob
from sklearn.model_selection import train_test_split

# Define the paths for the source images and labels
source_images_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/test/images'
source_labels_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/test/labels'

# Define the paths for the destination train and validation directories
train_images_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/train/images'
train_labels_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/train/labels'
valid_images_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/valid/images'
valid_labels_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/valid/lables'

# Create the destination directories if they don't exist
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(valid_images_dir, exist_ok=True)
os.makedirs(valid_labels_dir, exist_ok=True)

# Get a list of image files in the source images directory
image_files = glob(os.path.join(source_images_dir, '*.jpg'))  # Change '*.jpg' to match your image file types

# Split the files into training and validation sets (70-30 split)
train_files, valid_files = train_test_split(image_files, test_size=0.3, random_state=42)

# Function to move files to their new directories
def move_files(file_list, source_dir, dest_images_dir, dest_labels_dir):
    for file_path in file_list:
        base_name = os.path.basename(file_path)
        label_file_name = os.path.splitext(base_name)[0] + '.txt'  # Change '.txt' to your label file extension
        label_path = os.path.join(source_dir, label_file_name)

        # Move the image
        shutil.move(file_path, dest_images_dir)
        
        # Move the corresponding label file
        shutil.move(label_path, dest_labels_dir)

# Move the files to the respective train and valid directories
move_files(train_files, source_labels_dir, train_images_dir, train_labels_dir)
move_files(valid_files, source_labels_dir, valid_images_dir, valid_labels_dir)


In [ ]:
import os
import shutil
from glob import glob
from sklearn.model_selection import train_test_split

# Define the paths for the source images and labels
source_images_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/test/images'
source_labels_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/test/labels'

# Define the paths for the destination train and validation directories
train_images_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/train/images'
train_labels_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/train/labels'
valid_images_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/valid/images'
valid_labels_dir = '/home/buddhadev/Buddhadev_Everything/OPMD/valid/lables'

# Create the destination directories if they don't exist
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(valid_images_dir, exist_ok=True)
os.makedirs(valid_labels_dir, exist_ok=True)

# Get a list of image files in the source images directory
image_files = glob(os.path.join(source_images_dir, '*.jpg'))  # Change '*.jpg' to match your image file types

# Debugging line to print out the full path being accessed
print(f"Accessing directory: {os.path.join(source_images_dir, '*.jpg')}")

# Get a list of image files in the source images directory
image_files = glob(os.path.join(source_images_dir, '*.jpg'))  # Change '*.jpg' to match your image file types

# Only proceed if image files were found
if image_files:
    # Split the files into training and validation sets (70-30 split)
    train_files, valid_files = train_test_split(image_files, test_size=0.3, random_state=42)

    # Function to move files to their new directories
    def move_files(file_list, source_dir, dest_images_dir, dest_labels_dir):
        for file_path in file_list:
            base_name = os.path.basename(file_path)
            label_file_name = os.path.splitext(base_name)[0] + '.txt'  # Change '.txt' to your label file extension
            label_path = os.path.join(source_dir, label_file_name)

            # Move the image
            shutil.move(file_path, dest_images_dir)
            
            # Move the corresponding label file
            shutil.move(label_path, dest_labels_dir)

    # Move the files to the respective train and valid directories
    move_files(train_files, source_labels_dir, train_images_dir, train_labels_dir)
    move_files(valid_files, source_labels_dir, valid_images_dir, valid_labels_dir)
else:
    print("No image files found. Please check the directory and file type.")


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the path to the directory containing images and labels
data_directory = '/home/buddhadev/Buddhadev_Everything/OPMD/YOLO_OPMD_SP/'
image_directory = os.path.join(data_directory, 'images')
label_directory = os.path.join(data_directory, 'labels')

# Create directories for the training and validation splits
train_image_dir = os.path.join(data_directory, 'train_images')
train_label_dir = os.path.join(data_directory, 'train_labels')
valid_image_dir = os.path.join(data_directory, 'valid_images')
valid_label_dir = os.path.join(data_directory, 'valid_labels')

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(valid_image_dir, exist_ok=True)
os.makedirs(valid_label_dir, exist_ok=True)

# Verify the directory exists and list files
if os.path.exists(image_directory) and os.path.isdir(image_directory):
    print("Image directory exists:", image_directory)
    all_files = os.listdir(image_directory)
    print(f"All files in directory: {all_files}")  # This will print all files in the directory
    image_files = [f for f in all_files if f.endswith(('.jpg', '.png'))]
    print(f"Found {len(image_files)} image files.")
else:
    print("Image directory does not exist or is not a directory:", image_directory)

# Continue with the rest of the code if image files are found
if image_files:
    # Split the file names into training and validation sets
    train_files, valid_files = train_test_split(image_files, test_size=0.2, random_state=42)

    # Function to copy image and label files to the respective directories
    def copy_files(file_list, source_img_dir, dest_img_dir, source_lbl_dir, dest_lbl_dir):
        for image_file in file_list:
            base_filename = os.path.splitext(image_file)[0]
            label_filename = base_filename + '.txt'

            # Copy image
            shutil.copy2(os.path.join(source_img_dir, image_file), os.path.join(dest_img_dir, image_file))
            # Copy corresponding label
            shutil.copy2(os.path.join(source_lbl_dir, label_filename), os.path.join(dest_lbl_dir, label_filename))

    # Copy files to the train and validation directories
    copy_files(train_files, image_directory, train_image_dir, label_directory, train_label_dir)
    copy_files(valid_files, image_directory, valid_image_dir, label_directory, valid_label_dir)

    print(f"Training images and labels copied: {len(train_files)}")
    print(f"Validation images and labels copied: {len(valid_files)}")
else:
    print("No image files found, please check the directory and file permissions.")
